In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob

In [35]:

# 時間だけKingから持ってくる
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})

In [36]:
df_king.columns = ['DateTime', 'intra_price(円/kWh)', 'spot_price(円/kWh)',
       'gap_price[intra-spot](円/kWh)', 'intra_volume（MWh/h）',
       'spot_volume(kWh)', '北電太陽光想定(kWh)', '北電風力想定(kWh)', '北電太陽光実績(kWh)',
       '北電風力実績(kWh)', '九電太陽光想定(kWh)', '九電太陽光実績(kWh)', '九電風力想定(kWh)',
       '九電風力実績(kWh)', '東電太陽光想定(kWh)', '東電太陽光実績(kWh)', '東電風力想定(kWh)',
       '東電風力実績(kWh)', '関電太陽光想定(kWh)', '関電太陽光実績(kWh)', '関電風力想定(kWh)',
       '関電風力実績(kWh)', '中国太陽光想定(kWh)', '中国風力想定(kWh)', '中国太陽光実績(kWh)',
       '中国風力実績(kWh)', '北海道予備率(%)', '東北予備率(%)', '東京予備率(%)', '中部予備率(%)',
       '北陸予備率(%)', '関西予備率(%)', '中国予備率(%)', '四国予備率(%)', '九州予備率(%)', '認可出力合計',
       '低下量合計', '停止・低下ユニット数合計']

In [37]:
edit_cols = ['北電太陽光想定(kWh)', '北電風力想定(kWh)', '北電太陽光実績(kWh)','北電風力実績(kWh)']
for name in edit_cols:

    df_king[f'{name}'] = df_king[f'{name}']*1000/2
    

In [38]:
elcompany_cols = ['北電','九電','東電','関電','中国']

for error_name in elcompany_cols:

    df_king[f'{error_name}太陽光error(実績-想定)(kWh)'] = df_king[f'{error_name}太陽光実績(kWh)'] - df_king[f'{error_name}太陽光想定(kWh)']
    df_king[f'{error_name}風力error(実績-想定)(kWh)'] = df_king[f'{error_name}風力実績(kWh)'] - df_king[f'{error_name}風力想定(kWh)']

In [39]:
elcompany_cols = ['北電','九電','東電','関電','中国']

for zettai_error in elcompany_cols:
    df_king[f'絶対値{zettai_error}太陽光error(実績-想定)(kWh)'] = np.abs(df_king[f'{zettai_error}太陽光error(実績-想定)(kWh)'])
    df_king[f'絶対値{zettai_error}風力error(実績-想定)(kWh)'] = np.abs(df_king[f'{zettai_error}風力error(実績-想定)(kWh)'])

In [44]:
df_king.columns

Index(['DateTime', 'intra_price(円/kWh)', 'spot_price(円/kWh)',
       'gap_price[intra-spot](円/kWh)', 'intra_volume（MWh/h）',
       'spot_volume(kWh)', '北電太陽光想定(kWh)', '北電風力想定(kWh)', '北電太陽光実績(kWh)',
       '北電風力実績(kWh)', '九電太陽光想定(kWh)', '九電太陽光実績(kWh)', '九電風力想定(kWh)',
       '九電風力実績(kWh)', '東電太陽光想定(kWh)', '東電太陽光実績(kWh)', '東電風力想定(kWh)',
       '東電風力実績(kWh)', '関電太陽光想定(kWh)', '関電太陽光実績(kWh)', '関電風力想定(kWh)',
       '関電風力実績(kWh)', '中国太陽光想定(kWh)', '中国風力想定(kWh)', '中国太陽光実績(kWh)',
       '中国風力実績(kWh)', '北海道予備率(%)', '東北予備率(%)', '東京予備率(%)', '中部予備率(%)',
       '北陸予備率(%)', '関西予備率(%)', '中国予備率(%)', '四国予備率(%)', '九州予備率(%)', '認可出力合計',
       '低下量合計', '停止・低下ユニット数合計', '北電太陽光error(実績-想定)(kWh)',
       '北電風力error(実績-想定)(kWh)', '九電太陽光error(実績-想定)(kWh)',
       '九電風力error(実績-想定)(kWh)', '東電太陽光error(実績-想定)(kWh)',
       '東電風力error(実績-想定)(kWh)', '関電太陽光error(実績-想定)(kWh)',
       '関電風力error(実績-想定)(kWh)', '中国太陽光error(実績-想定)(kWh)',
       '中国風力error(実績-想定)(kWh)', '絶対値北電太陽光error(実績-想定)(kWh)',
       '絶対値北電風力error(実績-想定)

In [58]:
# それぞれの合算値と絶対値の合算値を作る。
df_king['太陽光誤差合算(実績-想定)(kWh)(ならし)'] = 0
df_king['風力誤差合算(実績-想定)(kWh)(ならし)'] =0
df_king['太陽光絶対値誤差合算(実績-想定)(kWh)'] = 0
df_king['風力絶対値誤差合算(実績-想定)(kWh)'] =0

array1 = df_king['太陽光誤差合算(実績-想定)(kWh)(ならし)'].values
elcompany_cols2 = ['九電','東電','関電','中国']
for name in elcompany_cols2:
    array1= array1 + df_king[f'{name}太陽光error(実績-想定)(kWh)'].values
df_king['太陽光誤差合算(実績-想定)(kWh)(ならし)'] = array1

array2 = df_king['風力誤差合算(実績-想定)(kWh)(ならし)'].values

for name in elcompany_cols2:
    array2= array2 + df_king[f'{name}風力error(実績-想定)(kWh)'].values
df_king['風力誤差合算(実績-想定)(kWh)(ならし)'] = array2


array3 = df_king['太陽光絶対値誤差合算(実績-想定)(kWh)'].values
for name in elcompany_cols2:
    array3= array3 + df_king[f'絶対値{name}太陽光error(実績-想定)(kWh)'].values
df_king['太陽光絶対値誤差合算(実績-想定)(kWh)'] = array3



array4 = df_king['風力絶対値誤差合算(実績-想定)(kWh)'].values
for name in elcompany_cols2:
    array4= array4 + df_king[f'絶対値{name}風力error(実績-想定)(kWh)'].values
df_king['風力絶対値誤差合算(実績-想定)(kWh)'] = array4



In [78]:
# 広域予備率を合算
df_king['広域予備率平均値(%)'] = 0

yobiritsu_array  = df_king['広域予備率平均値(%)'].values

cols_yobiritsu = ['北海道予備率(%)', '東北予備率(%)', '東京予備率(%)', '中部予備率(%)','北陸予備率(%)', '関西予備率(%)', '中国予備率(%)', '四国予備率(%)', '九州予備率(%)']
for yobi_name in cols_yobiritsu:
    yobiritsu_array = yobiritsu_array + df_king[f'{yobi_name}'].values
df_king['広域予備率平均値(%)'] = yobiritsu_array/9

In [82]:
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)
df_king.to_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',index=False)

In [77]:
pd.DataFrame(yobiritsu_array).dropna()

,0
70447,192.15
70448,205.38
70449,204.06
70450,200.34
70451,196.47
...,...
84170,262.67
84171,273.99
84172,298.31
84173,277.22


In [79]:
df_king.dropna()

,DateTime,intra_price(円/kWh),spot_price(円/kWh),gap_price[intra-spot](円/kWh),intra_volume（MWh/h）,spot_volume(kWh),北電太陽光想定(kWh),北電風力想定(kWh),北電太陽光実績(kWh),北電風力実績(kWh),...,絶対値東電風力error(実績-想定)(kWh),絶対値関電太陽光error(実績-想定)(kWh),絶対値関電風力error(実績-想定)(kWh),絶対値中国太陽光error(実績-想定)(kWh),絶対値中国風力error(実績-想定)(kWh),太陽光誤差合算(実績-想定)(kWh)(ならし),風力誤差合算(実績-想定)(kWh)(ならし),太陽光絶対値誤差合算(実績-想定)(kWh),風力絶対値誤差合算(実績-想定)(kWh),広域予備率平均値(%)
70447,2022-03-21 00:00:00,26.04,26.69,-0.65,1049.2,14127100,0.0,30400.0,0.0,19200.0,...,13306.0,0.0,3149.0,0.0,5208.0,0.0,32589.277610,0.0,43005.277610,21.350000
70448,2022-03-21 00:30:00,24.71,23.54,1.17,1016.3,14175050,0.0,30600.0,0.0,18100.0,...,8537.0,0.0,2591.0,0.0,5133.0,0.0,19787.346911,0.0,30053.346911,22.820000
70449,2022-03-21 01:00:00,25.03,23.54,1.49,1051.3,14276500,0.0,30300.0,0.0,14850.0,...,5793.0,0.0,5358.0,0.0,5287.0,0.0,20575.836705,0.0,31149.836705,22.673333
70450,2022-03-21 01:30:00,24.96,23.91,1.05,1088.2,14363200,0.0,30450.0,0.0,18200.0,...,10795.0,0.0,4839.0,0.0,5526.0,0.0,24819.836705,0.0,35871.836705,22.260000
70451,2022-03-21 02:00:00,26.46,23.91,2.55,1062.8,14340250,0.0,31150.0,0.0,14550.0,...,9837.0,0.0,5178.0,0.0,4997.0,0.0,21245.889875,0.0,31239.889875,21.830000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79754,2022-09-30 21:30:00,18.08,20.00,-1.92,787.4,17410950,0.0,35600.0,0.0,37550.0,...,33092.0,0.0,0.0,0.0,9112.0,0.0,23403.852764,0.0,42780.147236,32.960000
79755,2022-09-30 22:00:00,20.13,20.00,0.13,1323.0,16709950,0.0,36650.0,0.0,39800.0,...,30170.0,0.0,14.0,0.0,6527.0,0.0,23885.208319,0.0,36939.208319,33.890000
79756,2022-09-30 22:30:00,17.88,17.31,0.57,897.1,16285700,0.0,37650.0,0.0,41000.0,...,30703.0,0.0,99.0,0.0,11174.0,0.0,19987.191793,0.0,42335.191793,34.650000
79757,2022-09-30 23:00:00,17.35,17.06,0.29,933.0,16031850,0.0,38500.0,0.0,42300.0,...,24583.0,0.0,182.0,0.0,13027.0,0.0,12554.958776,0.0,38608.958776,34.911111


In [62]:
df_king.columns

Index(['DateTime', 'intra_price(円/kWh)', 'spot_price(円/kWh)',
       'gap_price[intra-spot](円/kWh)', 'intra_volume（MWh/h）',
       'spot_volume(kWh)', '北電太陽光想定(kWh)', '北電風力想定(kWh)', '北電太陽光実績(kWh)',
       '北電風力実績(kWh)', '九電太陽光想定(kWh)', '九電太陽光実績(kWh)', '九電風力想定(kWh)',
       '九電風力実績(kWh)', '東電太陽光想定(kWh)', '東電太陽光実績(kWh)', '東電風力想定(kWh)',
       '東電風力実績(kWh)', '関電太陽光想定(kWh)', '関電太陽光実績(kWh)', '関電風力想定(kWh)',
       '関電風力実績(kWh)', '中国太陽光想定(kWh)', '中国風力想定(kWh)', '中国太陽光実績(kWh)',
       '中国風力実績(kWh)', '北海道予備率(%)', '東北予備率(%)', '東京予備率(%)', '中部予備率(%)',
       '北陸予備率(%)', '関西予備率(%)', '中国予備率(%)', '四国予備率(%)', '九州予備率(%)', '認可出力合計',
       '低下量合計', '停止・低下ユニット数合計', '北電太陽光error(実績-想定)(kWh)',
       '北電風力error(実績-想定)(kWh)', '九電太陽光error(実績-想定)(kWh)',
       '九電風力error(実績-想定)(kWh)', '東電太陽光error(実績-想定)(kWh)',
       '東電風力error(実績-想定)(kWh)', '関電太陽光error(実績-想定)(kWh)',
       '関電風力error(実績-想定)(kWh)', '中国太陽光error(実績-想定)(kWh)',
       '中国風力error(実績-想定)(kWh)', '絶対値北電太陽光error(実績-想定)(kWh)',
       '絶対値北電風力error(実績-想定)

In [25]:
np.abs(df_king[f'{error_name}太陽光error(実績-想定)(kWh)'])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
84410    NaN
84411    NaN
84412    NaN
84413    NaN
84414    NaN
Name: 中国太陽光error(実績-想定)(kWh), Length: 84415, dtype: float64

In [23]:
df_king['九電太陽光想定(kWh)'].values[100:1500]

array([    0.,     0.,     0., ...,     0.,     0., 15051.])